In [ ]:
kf = KFold(n_splits=5, shuffle=True)
df_res = pd.DataFrame(dtype = 'object')

dict_param_grid = {'KNN': {'n_neighbors': [1,2,3,5,7,9,11]}}

In [ ]:
# extracting parameters
reg = KNeighborsRegressor()
name_reg = 'KNN'
param_grid = {str(reg)[:-2].lower()+'__'+k : v \
              for k,v in dict_param_grid[name_reg].items()}

In [ ]:
# researching best hyperparameters and fitting on training set
pipe = make_pipeline(column_trans, reg)
gscv = GridSearchCV(pipe,
                    param_grid=param_grid,
                    cv=kf, verbose=1)
gscv.fit(X_tr,y1_tr)

# best hyperparams
df_res = pd.DataFrame(dtype = 'object')
df_res.loc['name_params', name_reg] = tuple([k for k,v in dict_param_grid[name_reg].items()])
df_res.loc['best_params', name_reg] = tuple([gscv.best_params_[p] for p in param_grid])

# score of the model with best params on testing set
y_pr = gscv.predict(X_te)
res = scores_reg(name_reg, X_te, y1_te, y_pr).astype('object')
df_res = df_res.append(res.to_frame())

# mean cv score of the model with best params on testing set
res = cv_scores_reg(name_reg, gscv.best_estimator_, X_te, y1_te, cv=6).astype('object')
df_res = df_res.append(res.to_frame())
df_res

In [ ]:
# plotting optimisation curve of hyperparameters


list_n_params = [n for n in gscv_res.keys()if 'param_' in n]
list_params = [gscv.cv_results_[p].tolist() for p in list_n_params]
for n_p, li_p in zip(list_n_params, list_params):
    plt.errorbar(li_p,
            gscv.cv_results_['mean_test_score'],
            yerr=gscv.cv_results_['std_test_score'])
    plt.gcf().suptitle(n_p)
# plt.gca().set_xscale('log')
# plt.gca().set(xlim=(0,10000))

In [ ]:
reg = LinearRegression()
name_reg = 'LinReg(allcols)'

pipe = make_pipeline(column_trans, reg)
pipe.fit(X_tr,y1_tr)

y_pr = pipe.predict(X_te)

res = scores_reg(name_reg, X_te, y1_te, y_pr).astype('object')
df_lr = df_lr.append(res.to_frame())
res = cv_scores_reg(name_reg, pipe,
                    X_te, y1_te, cv=6).astype('object')
df_lr = df_lr.append(res.to_frame())

df_lr

In [ ]:
# plotting optimisation curve of hyperparameters
param_grid = dict_param_grid[name_reg]

gscv_res = knn_gscv.cv_results_
list_n_params = [n for n in gscv_res.keys()if 'param_' in n]
list_params = [gscv_res[p].tolist() for p in list_n_params]
for n_p, li_p in zip(list_n_params, list_params):
    plt.errorbar(li_p,
            gscv_res['mean_test_score'],
            yerr=gscv_res['std_test_score'])
    plt.gcf().suptitle(n_p)
# plt.gca().set_xscale('log')
# plt.gca().set(xlim=(0,10000))

In [ ]:
reg = ElasticNet()
name_reg = 'my_enet'
dict_param_grid = {'my_enet': {'alpha': [1,15],
                               'l1_ratio': [0.3,0.5]}}
kf_gscv = KFold(n_splits=2, shuffle=True)

# version unique
enet_gscv, new_df_res = model_optimizer(name_reg=name_reg, data_preproc=column_trans,
                                 reg=reg, param_grid=dict_param_grid[name_reg],
                                 Xtr=X_tr, ytr=y1_tr, Xte=X_te, yte=y1_te,
                                 cv_gs=kf_gscv, cv_test=6)
df_res = pd.concat([df_res, new_df_res], axis=1)

display(df_res)

plot_2D_hyperparam_opt(enet_gscv)

In [ ]:
reg = LinearRegression()
name_reg = 'LinReg(allcols)'

pipe = make_pipeline(column_trans, reg)
pipe.fit(X_tr,y1_tr)

y_pr = pipe.predict(X_te)

ser = scores_reg(name_reg, X_te_sel, y1_te, y_pr).astype('object')
ser = ser.append(cv_scores_reg(name_reg, pipe, X_te_sel, y1_te, cv=6).astype('object'))

df_lr = df_lr.append(ser.to_frame())

df_lr

In [ ]:
# name_reg = name_reg
# pipe= pipe
# Xte = X_te
# yte = y1_te
# df_res=df_lr
# cv=5

# if df_res is None:
#     df_res = pd.DataFrame(dtype = 'object')
# df_res_mod = pd.DataFrame(dtype = 'object')
# ypr = pipe.predict(Xte)
# ser = scores_reg(name_reg, Xte, yte, ypr).astype('object')
# ser = ser.append(cv_scores_reg(name_reg, pipe, Xte, yte, cv=cv).astype('object'))
# df_res_mod = pd.concat([df_res,ser.to_frame()],1)
# df_res_mod

In [ ]:
def plot_append_curve(x, y, c, fig=None):
    %matplotlib
    if fig is None:
        fig = plt.figure()
    ax = fig.axes[0]
    ax.plot(x, y, c=c)
    %matplotlib inline
    return fig

fig = plot_append_curve([100,11,90,13], [100,90,70,200], c='grey', fig=fig);
fig

In [ ]:
# OLD VERSION
class CustEncoder(BaseEstimator) :

    def __init__(self, l_card_cols, h_card_cols, bool_cols,
                 strat_low_card='ord', strat_high_card='ord'):
        self.l_card_cols = l_card_cols
        self.h_card_cols = h_card_cols
        self.bool_cols = bool_cols
        self.strat_low_card = strat_low_card
        self.strat_high_card = strat_high_card

    def fit(self, X, y=None):
        # Dictionary to translate strategies
        d_enc = {'ohe': ce.OneHotEncoder(),
                 'hash': ce.HashingEncoder(),
                 'ord': ce.OrdinalEncoder(),
                 'loo': ce.LeaveOneOutEncoder(),
                 'bin': ce.BinaryEncoder()}

        # Creates a columns transformer with chosen strategies
        self.column_trans = ColumnTransformer([("low_card", d_enc[self.strat_low_card], l_card_cols),
                                       ("bool", ce.OrdinalEncoder(), bool_cols),
                                       ("high_card", d_enc[self.strat_high_card], h_card_cols)],
                                       remainder=StandardScaler())
        return self.column_trans.fit(X, y)
  
    def transform(self, X, y=None):
        return  self.column_trans.transform(X)

In [ ]:
results = knn_gscv3.cv_results_
scoring = ['neg_root_mean_squared_error', 'r2']
param = 'param_KNN3__n_neighbors'


plt.figure(figsize=(8, 5))
plt.title("GridSearchCV evaluating using multiple scorers simultaneously",
          fontsize=16)
plt.xlabel("n_neighbors")
plt.ylabel("Score")
ax = plt.gca()

# Get the regular numpy array from the MaskedArray
X_axis = np.array(results[param].data, dtype=float)

for scorer, color in zip(sorted(scoring), ['g', 'k']):
    for sample, style in (('train', '--'), ('test', '-')):
        sample_score_mean = results['mean_%s_%s' % (sample, scorer)]
        sample_score_std = results['std_%s_%s' % (sample, scorer)]
        ax.fill_between(X_axis, sample_score_mean - sample_score_std,
                        sample_score_mean + sample_score_std,
                        alpha=0.1 if sample == 'test' else 0, color=color)
        ax.plot(X_axis, sample_score_mean, style, color=color,
                alpha=1 if sample == 'test' else 0.7,
                label="%s (%s)" % (scorer, sample))

    best_index = np.nonzero(results['rank_test_%s' % scorer] == 1)[0][0]
    best_score = results['mean_test_%s' % scorer][best_index]

    # Plot a dotted vertical line at the best score for that scorer marked by x
    ax.plot([X_axis[best_index], ] * 2, [0, best_score],
            linestyle='-.', color=color, marker='x', markeredgewidth=3, ms=8)

    # Annotate the best score for that scorer
    ax.annotate("%0.2f" % best_score, 
                (X_axis[best_index]+0.5, best_score + 0.005))

plt.legend(loc="best")
plt.grid(False)
plt.show()